<a href="https://www.kaggle.com/code/jorgeluiscruzquispe/valley-classification?scriptVersionId=162224445" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# <center>Valley Classification</center>
---

In [1]:
!pip install geemap

  Obtaining dependency information for geemap from https://files.pythonhosted.org/packages/c8/fb/41d082fdf37ead7312142d18a8d0138f48dc6c581ee483804e74dbc2a786/geemap-0.30.4-py2.py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 9.9 MB/s eta 0:00:00
  Obtaining dependency information for ipyevents from https://files.pythonhosted.org/packages/af/49/5692d363a82838282f5c33a787922f07115d7ba4b452391e51db95c9d247/ipyevents-2.0.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for python-box from https://files.pythonhosted.org/packages/65/63/c2df733c7e9549c1acc6919a3157f8f2fe833fa013052b169e228e1aeb75/python_box-7.1.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for scooby from https://files.pythonhosted.org/packages/df/7b/25af5b3327c23ec3053f54a50b120679baf319f5d67047203cf

In [2]:
import geemap.foliumap as geemap
import geemap.colormaps as cmap
import ee
import os

In [3]:
# Define a map centered on San Francisco.
Map = geemap.Map()

roi = ee.Geometry.Point([-77.739944,-10.703792]).buffer(25000)

# Create a circle by drawing a 20000 meter buffer around a point.
landsat8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')\
                                .filterDate('2022-01-01', '2022-03-31')\
                                .select(['B[1-7]'])\
                                .filterMetadata('CLOUD_COVER_LAND', 'less_than', 25)

# realizando recorte
landsat_viz = ee.Image(landsat8.median())
landsat_clip = landsat_viz.clip(roi)

# Create an NDWI image, define visualization parameters and display.
ndvi = landsat_clip.normalizedDifference(['B5', 'B4'])
ndvi_viz = {'min': 0.1, 'max': 0.45, 'palette': cmap.palettes.ndvi}

# Mask the non-watery parts of the image, where NDWI < 0.4.
ndvi_masked = ndvi.updateMask(ndvi.gte(0.2))

Map.centerObject(roi, 11)

Map.add_basemap("HYBRID")

# Add the image layer to the map and display it.
Map.add_layer(landsat_clip, {'bands': ['B4', 'B5', 'B3']}, 'imagen satelital en color natural', opacity = 1, shown = True)
Map.add_layer(ndvi_masked, ndvi_viz, 'ndvi de la zona de estudio', opacity = 1, shown = False)
Map.add_layer(ee.Image().paint(roi, 0, 2), {'palette': 'red'}, 'area de estudio', opacity =1)
Map

Enter verification code:  4/1AfJohXnyTGt2N79v7YgZypQAgCsLnZ85bNT5uQ36Z5_B3cEzNgC5OwBVHG4



Successfully saved authorization token.


In [6]:
Map = geemap.Map()

props = geemap.image_props(landsat_clip)

# Make the training dataset.
training = landsat_clip.sample(
    **{
        #     'region': region,
        'scale': 30,
        'numPixels': 5000,
        'seed': 0,
        'geometries': True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(training, {}, 'training', False)

# Instantiate the clusterer and train it.
n_clusters = 4
clusterer = ee.Clusterer.wekaKMeans(n_clusters).train(training)

# Cluster the input using the trained clusterer.
result = landsat_clip.cluster(clusterer)

legend_keys = ['clasificación 1', 'clasificación 2', 'clasificación 3', 'clasificación 4']
legend_colors = ['#42f54e', '#0e0eeb', '#f20a0a', '#faf200']

# Reclassify the map
result = result.remap([0, 1, 2, 3], [1, 2, 3, 4])

Map.addLayer(result.randomVisualizer(), {}, 'clusters', shown = False)
Map.addLayer(result, {'min': 1, 'max': 5, 'palette': legend_colors}, 'Labelled clusters')
Map.add_layer(ndvi_masked, ndvi_viz, 'ndvi del valle de pativilca', opacity = 1, shown= False)
Map.add_layer(ee.Image().paint(roi, 0, 2), {'palette': 'red'}, 'area de estudio', opacity =1)

Map.add_legend(title = "leyenda", labels=legend_keys, colors=legend_colors, position='bottomright')

Map.centerObject(roi, 11)

download_dir = os.path.normpath(os.getcwd())
if not os.path.exists(download_dir):
    os.makedirs(download_dir)
html_file = os.path.join(download_dir, 'my_map.html')
Map.to_html(filename=html_file, title='My Map', width='100%', height='880px')

Map

In [70]:
Map = geemap.Map()

zones = ee.Image(0).where(result.gt(1), 1).where(result.gt(2), 2).where(result.gt(3), 3).where(result.gt(4), 4)

zones = zones.updateMask(result.gt(0)).rename('zone')

projection = result.projection()

scale = result.projection().nominalScale()

Map.addLayer(zones, {"palette": cmap.palettes.Greens}, "class image")

Map.centerObject(roi, 11)

ClassVector = zones.reduceToVectors(
    geometry = roi,
    crs = projection,
    scale = 30, # scale
    geometryType = 'polygon',
    eightConnected = False,
    labelProperty = 'zone',
    bestEffort = False,
    maxPixels = 1e13,
    tileScale = 3 # In case of error.
)

ClassDrawn = ClassVector.draw(color = 'blue',strokeWidth = 2)

Map.addLayer(ClassDrawn, {}, 'class vector', opacity = 0.5)

Map